# 🧪 Data Mining Project Lab: Malaria Outbreak Prediction in Cameroon

In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn geopandas plotly

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached plotly-6.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached pyogrio-0.11.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached pyproj-3.6.1-cp39-cp39-win_amd64.whl.metadata (31 kB)
  Using cached shapely-2.0.7-cp39-cp39-win_amd64.whl.metadata (7.1 kB)
  Using cached narwhals-1.41.0-py3-none-any.whl.metadata (11 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached plotly-6.1.2-py3-none-any.whl (16.3 MB)
Using cached narwhals-1.41.0-py3-none-any.whl (357 kB)
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.2 MB 1.9 MB/s eta 0:00:11
   -- ------------------------------------- 1.0/19.2 MB 2.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/19.2 MB 2.4 MB/s et


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\JA 12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\JA 12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\JA 12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\JA 12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3

In [9]:
!pip install pyogrio


  Using cached pyogrio-0.11.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.2 MB 289.3 kB/s eta 0:01:05
   - -------------------------------------- 0.5/19.2 MB 289.3 kB/s eta 0:01:05
   - -------------------------------------- 0.5/19.2 MB 289.3 kB/s eta 0:01:05
   - -------------------------------------- 0.5/19.2 MB 289.3


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install geopandas



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyproj-3.6.1-cp39-cp39-win_amd64.whl.metadata (31 kB)
  Using cached shapely-2.0.7-cp39-cp39-win_amd64.whl.metadata (7.1 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.1 MB 441.3 kB/s eta 0:00:13
   --- ------------------------------------ 0.5/6.1 MB 441.3 kB/s eta 0:00:13
   --- ------------------------------------ 0.5/6.1 MB 441.3 kB/s eta 0:00:13
   ----- ------------

In [12]:
!python -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- -------------------------

In [13]:
!pip install plotly


  Using cached plotly-6.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached narwhals-1.41.0-py3-none-any.whl.metadata (11 kB)
Using cached plotly-6.1.2-py3-none-any.whl (16.3 MB)
Using cached narwhals-1.41.0-py3-none-any.whl (357 kB)

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwha

## 📁 Step 1: Data Collection

In [2]:
import pandas as pd

# Load CSV
df = pd.read_csv("RELAY_WHS.csv")

# Preview
df.head()

,IND_ID,IND_CODE,IND_UUID,IND_PER_CODE,DIM_TIME,DIM_TIME_TYPE,DIM_GEO_CODE_M49,DIM_GEO_CODE_TYPE,DIM_PUBLISH_STATE_CODE,IND_NAME,GEO_NAME_SHORT,RATE_PER_1000_N,RATE_PER_1000_NL,RATE_PER_1000_NU
0,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,600,COUNTRY,NaN,Malaria cases,Paraguay,NaN,NaN,NaN
1,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,598,COUNTRY,NaN,Malaria cases,Papua New Guinea,NaN,NaN,NaN
2,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,591,COUNTRY,NaN,Malaria cases,Panama,NaN,NaN,NaN
3,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,586,COUNTRY,NaN,Malaria cases,Pakistan,NaN,NaN,NaN
4,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,566,COUNTRY,NaN,Malaria cases,Nigeria,NaN,NaN,NaN


In [3]:
cameroon_df = df[df["GEO_NAME_SHORT"] == "Cameroon"].copy()
cameroon_df.shape

(24, 14)

## 🧼 Step 2: Data Preprocessing

In [5]:
# Check for missing values
print(cameroon_df.isnull().sum())

# Drop or fill missing
cameroon_df.fillna(method='ffill', inplace=True)

# Convert date column
# Check if 'YEAR' column exists in the dataframe
if 'YEAR' in cameroon_df.columns:
    cameroon_df["YEAR"] = pd.to_datetime(cameroon_df["YEAR"], errors='coerce')
else:
    print("'YEAR' column not found in the dataframe")


IND_ID                     0
IND_CODE                   0
IND_UUID                   0
IND_PER_CODE               0
DIM_TIME                   0
DIM_TIME_TYPE              0
DIM_GEO_CODE_M49           0
DIM_GEO_CODE_TYPE          0
DIM_PUBLISH_STATE_CODE    24
IND_NAME                   0
GEO_NAME_SHORT             0
RATE_PER_1000_N           24
RATE_PER_1000_NL          24
RATE_PER_1000_NU          24
dtype: int64
'YEAR' column not found in the dataframe


C:\Users\JA 12\AppData\Local\Temp\ipykernel_14456\4159450963.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cameroon_df.fillna(method='ffill', inplace=True)


In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# Reshape data for MinMaxScaler which expects 2D array
cases = cameroon_df["Number_of_Cases"].values.reshape(-1, 1)
cameroon_df["Normalized_Cases"] = scaler.fit_transform(cases)


KeyError: 'Number_of_Cases'

## 📊 Step 3: Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.lineplot(data=cameroon_df, x="YEAR", y="Malaria_Cases_Per_1000")
plt.title("Malaria Incidence Over Time")

In [ ]:
threshold = cameroon_df["Malaria_Cases_Per_1000"].quantile(0.75)
cameroon_df["Outbreak"] = cameroon_df["Malaria_Cases_Per_1000"] > threshold

In [ ]:
corr = cameroon_df.select_dtypes(include='number').corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")

## 🧠 Step 4: Modeling

In [ ]:
from sklearn.model_selection import train_test_split

X = cameroon_df[["Rainfall_mm", "Humidity", "Temperature_C"]]
y = cameroon_df["Outbreak"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

## 🧪 Step 5: Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## 🚀 Step 6: Insights & Deployment

In [ ]:
importances = pd.Series(clf.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', title='Feature Importance')

In [ ]:
import joblib

joblib.dump(clf, "malaria_model.pkl")